In [1]:
import os 
import pandas as pd

from datetime import datetime
import random as rnd
import itertools as it

import warnings
warnings.filterwarnings('ignore')

# Making pics info DataFrame

In [ ]:
path = 'dataset/lfw_funneled'

pairs = ''
with open(f'{path}/pairs.txt', 'r') as file:
    pairs = file.read()

df = pd.DataFrame(list(map(lambda x: x.split('\t'), pairs.split('\n')[1:301])), columns=['name', 'pic1', 'pic2'])
df = (pd.melt(df, id_vars='name', value_vars=['pic1', 'pic2'], var_name='picNum', value_name='pic')
        .drop('picNum', axis=1)
        .sort_values(by=['name', 'pic'])
        .drop_duplicates()
        .reset_index(drop=True))

df['pic'] = df['pic'].apply(lambda x: x.zfill(4))
df['path_x1'] = str(path) + '/' + df['name'] + '/' + df['name'] + '_' + df['pic'] + '.jpg'
df.head()

In [ ]:
pics_to_scale_path = df['path_x1'].values
pics_to_scale = list(map(lambda x: x.split('/')[-1][:-4]+ '_result.jpg', pics_to_scale_path))
print(f'There are {len(pics_to_scale)} unique pics to be scaled')

# Scaling and adding path info to pics DataFrame

In [ ]:
# Тут надо самому ставить нужный скейл: 2 3 или 4. Так как памяти нехватает + настроить сохранение
# в нужную папку сложна (не хочется заморачиваться), то надо перезапускать и руками переименовывать папку.

scaled_pics = os.listdir('dataset/output')
for pic, pic_path in zip(pics_to_scale, pics_to_scale_path):
    if pic in scaled_pics:
        continue
    abc = f'--file={pic_path} --scale=4'
    %run sr.py {abc}

In [ ]:
for scale in [2, 3, 4]:
    df[f'path_x{scale}'] = f'dataset/output_x{scale}/'+df[f'path_x1'].apply(lambda x: x.split('/')[-1][:-4])+'_result.jpg'
df.head()

In [ ]:
df.to_csv('pics_df.csv')

# Making DataFrame for classification

In [ ]:
df = pd.read_csv('pics_df.csv', index_col=0)
df.head()

#### Получим всевозможные пары фоточек для каждого человека

In [ ]:
classification_data_sim = []
for name, name_df in df[['name', 'pic']].groupby('name'):
    pics = name_df['pic'].values
    pics_comb = list(it.combinations(pics, 2))
    pics_name_comb = list(map(lambda x: [name]+list(x), pics_comb))
    for comb in pics_name_comb:
        classification_data_sim.append(comb)
classification_df_sim = pd.DataFrame(classification_data_sim, columns=['name', 'pic1', 'pic2'])
classification_df_sim.head()

In [ ]:
for pic_num in [1, 2]:
    classification_df_sim = (pd.merge(classification_df_sim, df, 
                                      left_on=['name', f'pic{pic_num}'], right_on=['name', 'pic'], 
                                      how='left', suffixes=('_1', '_2')))
classification_df_sim.drop(['pic_1', 'pic_2'], axis=1, inplace=True)
classification_df_sim.head()

In [ ]:
classification_df_sim.to_csv('classification_similar_people_pics_df.csv')
print(f'We\'ve got {classification_df_sim.shape[0]} pairs of pics with similar persons')

#### И столько же для разных людей

In [ ]:
non_sim_people_pics = [el for el in it.combinations(df[['name', 'pic']].values.tolist(), 2) if el[0][0] != el[1][0]]
non_sim_people_pics = [el[0]+el[1] for el in non_sim_people_pics]
non_sim_people_pics = rnd.sample(non_sim_people_pics, classification_df_sim.shape[0])
classification_df_nonsim = pd.DataFrame(non_sim_people_pics, columns=['name1', 'pic1', 'name2', 'pic2'])
classification_df_nonsim.head()

In [ ]:
for pic_num in [1, 2]:
    classification_df_nonsim = (pd.merge(classification_df_nonsim, df, 
                                      left_on=[f'name{pic_num}', f'pic{pic_num}'], right_on=['name', 'pic'], 
                                      how='left', suffixes=('_1', '_2')))
classification_df_nonsim.drop(['name_1', 'pic_1', 'name_2', 'pic_2'], axis=1, inplace=True)
classification_df_nonsim.head()

In [ ]:
classification_df_nonsim.to_csv('classification_non-similar_people_pics_df.csv')
print(f'And {classification_df_nonsim.shape[0]} pairs of pics with non-similar persons')

# Classification

In [4]:
from recognizer import Recognizer
from skimage.io import imread

THRESHOLD = 0.9

In [5]:
recognizer = Recognizer()

def recongize(row):
    photo1 = imread(row.ix[0])
    photo2 = imread(row.ix[1])
    similarity = recognizer.get_best_similarity(photo1, photo2)
    return similarity < THRESHOLD

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from ./facenet_models/facenet/model-20180408-102900.ckpt-90


### Similar and non-similar people

In [6]:
data_similar = pd.read_csv('classification_similar_people_pics_df.csv', index_col=0)
data_nonsimilar = pd.read_csv('classification_non-similar_people_pics_df.csv', index_col=0)

In [7]:
%%time
print(f'Started at {datetime.now()}')
for data in [data_similar, data_nonsimilar]:
    for scale in [1, 2, 3, 4]:
        start = datetime.now()
        data[f'rec_x{scale}'] = data[[f'path_x{scale}_1', f'path_x{scale}_2']].apply(recongize, axis=1)
        print(f'Scale x{scale} is done in {datetime.now() - start}')

Started at 2019-05-02 23:39:42.658064
Scale x1 is done in 0:03:43.563873
Scale x2 is done in 0:08:56.335615
Scale x3 is done in 0:16:53.689556
Scale x4 is done in 0:27:20.152157
Scale x1 is done in 0:03:37.578908
Scale x2 is done in 0:08:45.188431
Scale x3 is done in 0:16:28.823655
Scale x4 is done in 0:27:13.626479
Wall time: 1h 52min 58s


In [8]:
data_similar.to_csv('classification_similar_people_pics_df.csv')
data_nonsimilar.to_csv('classification_non-similar_people_pics_df.csv')

In [32]:
def counts_pct(col):
    return round(col.value_counts(normalize=True) * 100, 2)

classification_results = pd.DataFrame()
classification_results['person_type'] = ['Similar'] * 4 + ['Non-Similar'] * 4 + ['Mixed'] * 4
classification_results['scale'] = ['x1', 'x2', 'x3', 'x4'] * 3

recs = ['rec_x1', 'rec_x2', 'rec_x3', 'rec_x4']
recs_sim = data_similar[recs]
recs_nonsim = data_nonsimilar[recs].replace({True: False, False: True})
classification_results['accuracy'] = (list(recs_sim.apply(counts_pct).loc[True].values) + 
                                      list(recs_nonsim.apply(counts_pct).loc[True].values) +
                                      list(pd.concat([recs_sim, recs_nonsim]).apply(counts_pct).loc[True].values))

classification_results

,person_type,scale,accuracy
0,Similar,x1,80.94
1,Similar,x2,83.21
2,Similar,x3,83.93
3,Similar,x4,83.81
4,Non-Similar,x1,65.59
5,Non-Similar,x2,61.27
6,Non-Similar,x3,60.91
7,Non-Similar,x4,60.91
8,Mixed,x1,73.26
9,Mixed,x2,72.24


In [33]:
classification_results.to_csv('classification_results_DCSCN.csv')